In [1]:
import os
import gc
import glob
import sys
import time
import random
import logging
import argparse
from datetime import datetime
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import tqdm

from sklearn.model_selection import train_test_split, KFold

import timm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, Subset
from torchvision import datasets, transforms
from PIL import Image

logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)


class ImageDataset(Dataset):
    def __init__(self, data_df, transform=None):

        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        # достаем имя изображения и ее лейбл
        image_name, label = self.data_df.iloc[idx]['ID_img'], self.data_df.iloc[idx]['class']

        # читаем картинку. read the image
        image = cv2.imread(f"../train/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно. transform it, if necessary
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label).long()
    
    def __len__(self):
        return len(self.data_df)


transform_valid = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

data = []
for cl in [0, 1, 2]:
    for image_name in glob.glob(f"../2906/{cl}/*.jpg"):
        data.append({"ID_img":Path(image_name).name, "class":cl})
    for image_name in glob.glob(f"../2906/{cl}/*.jpeg"):
        data.append({"ID_img":Path(image_name).name, "class":cl})
data_df = pd.DataFrame(data)

dataset    = ImageDataset(data_df, transform_valid)
dataloader = torch.utils.data.DataLoader(dataset=dataset,
                                       batch_size=2,
                                       shuffle=False,
                                       pin_memory=True,
                                       num_workers=2)

In [2]:
res = []
for models in glob.glob(f"checkpoints/*.pth"):
    print(models)
    torch.cuda.empty_cache()
    model = timm.models.swin_large_patch4_window12_384(pretrained=False)
    model.head = nn.Sequential(nn.Linear(model.head.weight.shape[1], 4096),
                    nn.Dropout(0.10),
                    nn.ReLU(),
                    nn.Linear(4096, 3))
    model = model.cuda()
    model.load_state_dict(torch.load(models))
    
    model.eval()
    
    preds = []

    with torch.no_grad():
        for imgs, labels in dataloader:

            imgs = imgs.cuda()
            labels = labels.cuda()

            pred = model(imgs)
            preds.extend(pred.cpu().numpy().tolist())
    res.append({'models': models, 'preds': preds})

checkpoints/000_2906.pth


/home/andyst/anaconda3/envs/lab/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


checkpoints/001_2906.pth
checkpoints/002_2906.pth
checkpoints/003_2906.pth
checkpoints/004_2906.pth
checkpoints/005_2906.pth
checkpoints/006_2906.pth


In [3]:
y_true = [x[1].item() for x in dataset]
y_true = torch.LongTensor(y_true)

In [4]:
out = []

for _ in range(1000):
    idxs = np.random.choice(list(range(len(res))), size=5, replace=False)

    model_list = []
    for i, idx in enumerate(idxs):
        model_list.append(res[idx]['models'])
        if i == 0:
            pred  = torch.sigmoid(torch.FloatTensor(res[idx]['preds']))
        else:
            pred += torch.sigmoid(torch.FloatTensor(res[idx]['preds']))

    pred = pred.argmax(1)
    acc = (y_true == pred).sum() / len(y_true)
    
    out.append((acc, model_list))

In [5]:
out = sorted(out, key=lambda x: -x[0])
out[:5]

[(tensor(1.),
  ['checkpoints/006_2906.pth',
   'checkpoints/000_2906.pth',
   'checkpoints/001_2906.pth',
   'checkpoints/004_2906.pth',
   'checkpoints/005_2906.pth']),
 (tensor(1.),
  ['checkpoints/002_2906.pth',
   'checkpoints/001_2906.pth',
   'checkpoints/004_2906.pth',
   'checkpoints/005_2906.pth',
   'checkpoints/003_2906.pth']),
 (tensor(1.),
  ['checkpoints/006_2906.pth',
   'checkpoints/002_2906.pth',
   'checkpoints/005_2906.pth',
   'checkpoints/004_2906.pth',
   'checkpoints/003_2906.pth']),
 (tensor(1.),
  ['checkpoints/005_2906.pth',
   'checkpoints/001_2906.pth',
   'checkpoints/004_2906.pth',
   'checkpoints/003_2906.pth',
   'checkpoints/002_2906.pth']),
 (tensor(1.),
  ['checkpoints/005_2906.pth',
   'checkpoints/002_2906.pth',
   'checkpoints/004_2906.pth',
   'checkpoints/006_2906.pth',
   'checkpoints/001_2906.pth'])]

In [6]:
test_df = pd.read_csv("../sample_solution.csv")
test_df = test_df.drop(["class"], axis = 1)

valid_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

class TestImageDataset(Dataset):
    def __init__(self, data_df, transform=None):
        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        image_name = self.data_df.iloc[idx]['ID_img']
        
        # читаем картинку
        image = cv2.imread(f"../test/{image_name}.jpg")
        if image is None:
            print(f"{image_name} is empty")
            return torch.zeros(3, 384, 384)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)

            # преобразуем, если нужно
            if self.transform:
                image = self.transform(image)

            return image
    
    def __len__(self):
        return len(self.data_df)


test_dataset = TestImageDataset(test_df, valid_transform)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=2,
                                           # shuffle=True,
                                           pin_memory=True,
                                           num_workers=1)

In [7]:
test_out = []
for i, modelname in enumerate(out[0][1]):
    torch.cuda.empty_cache()
    model = timm.models.swin_large_patch4_window12_384(pretrained=False)
    model.head = nn.Sequential(nn.Linear(model.head.weight.shape[1], 4096),
                    nn.Dropout(0.10),
                    nn.ReLU(),
                    nn.Linear(4096, 3))
    model = model.cuda()
    model.load_state_dict(torch.load(modelname))

    model.eval()
    pred = []
    with torch.no_grad():
        for imgs in test_loader:
            imgs = imgs.cuda()
            pred.append(model(imgs).cpu().detach().numpy())
            
    if i == 0:
        test_out  = np.vstack(pred)
    else:
        test_out += np.vstack(pred)


40567458052_99667140804_20508599954_6 is empty
40567458052_99667140804_20508599954_6 is empty
40567458052_99667140804_20508599954_6 is empty
40567458052_99667140804_20508599954_6 is empty
40567458052_99667140804_20508599954_6 is empty


In [8]:
predicts = np.argmax(test_out, axis=1)
test_names = test_df["ID_img"]

submit_df = pd.DataFrame([[name, pred] for name, pred in zip(test_names, predicts)], columns=['ID_img', 'class'])
submit_df

,ID_img,class
0,34020749806_42065966214_42113475048_2,2
1,80128313599_98196458454_79029076007_8,1
2,17820331238_48919943775_53688855463_7,2
3,70492442702_21083599816_22777758696_0,2
4,94790217016_17108156014_60668676818_2,2
...,...,...
220,60879177998_15763718934_82574532042_2,2
221,11758169966_65799840524_72283028069_1,1
222,9259096884_2251720133_44072689872_8,0
223,37732252922_9265441355_19052721018_3,1


In [9]:
submit_df.to_csv(f"timm_cv7_best5_2906v8.csv", index=False)